In [1]:
import sqlalchemy
from sqlalchemy import create_engine
sqlalchemy.__version__

'1.3.5'

In [50]:
engine = create_engine('mysql+mysqlconnector://root:@localhost:3306/test', echo=True)

In [45]:
#engine = create_engine('mysql+mysqlconnector://root:AI@2019@ai@rm-8vbwj6507z6465505ro.mysql.zhangbei.rds.aliyuncs.co:3306/stu_db')

In [46]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [47]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata = MetaData()
users = Table('users', metadata,
        Column('id', Integer, primary_key=True),
        Column('name', String(20)),
        Column('fullname', String(20)),
)

addresses = Table('addresses', metadata,
        Column('id', Integer, primary_key=True),
        Column('user_id', None, ForeignKey('users.id')),
        Column('email_address', String(30), nullable=False)
)

In [51]:
metadata.create_all(engine)

2019-08-19 06:22:28,812 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2019-08-19 06:22:28,814 INFO sqlalchemy.engine.base.Engine {}
2019-08-19 06:22:28,830 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2019-08-19 06:22:28,830 INFO sqlalchemy.engine.base.Engine {}
2019-08-19 06:22:28,835 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2019-08-19 06:22:28,835 INFO sqlalchemy.engine.base.Engine {}
2019-08-19 06:22:28,841 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2019-08-19 06:22:28,842 INFO sqlalchemy.engine.base.Engine {}
2019-08-19 06:22:28,845 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2019-08-19 06:22:28,846 INFO sqlalchemy.engine.base.Engine {}
2019-08-19 06:22:28,849 INFO sqlalchemy.engine.base.Engine DESCRIBE `users`
2019-08-19 06:22:28,850 INFO sqlalchemy.engine.base.Engine {}
2019-08-19 06:22:28,862 INFO sqlalchemy.engine.ba

In [15]:
ins = users.insert()

In [16]:
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

In [17]:
ins = users.insert().values(name='jack', fullname='Jack Jones')

In [18]:
str(ins)

'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

In [19]:
ins.compile().params

{'name': 'jack', 'fullname': 'Jack Jones'}

执行

In [20]:
conn = engine.connect()

conn

In [21]:
result = conn.execute(ins)

2019-08-18 22:57:22,010 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (%(name)s, %(fullname)s)
2019-08-18 22:57:22,012 INFO sqlalchemy.engine.base.Engine {'name': 'jack', 'fullname': 'Jack Jones'}
2019-08-18 22:57:22,020 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
ins.bind = engine

str(ins)

'INSERT INTO users (name, fullname) VALUES (%(name)s, %(fullname)s)'

In [23]:
result.inserted_primary_key

[1]

### Executing Multiple Statements

In [24]:
ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

2019-08-18 22:57:28,660 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (%(id)s, %(name)s, %(fullname)s)
2019-08-18 22:57:28,661 INFO sqlalchemy.engine.base.Engine {'id': 2, 'name': 'wendy', 'fullname': 'Wendy Williams'}
2019-08-18 22:57:28,663 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
conn.execute(addresses.insert(), [
    {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
    {'user_id': 1, 'email_address' : 'jack@msn.com'},
    {'user_id': 2, 'email_address' : 'www@www.org'},
    {'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

2019-08-18 22:57:29,497 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (%(user_id)s, %(email_address)s)
2019-08-18 22:57:29,499 INFO sqlalchemy.engine.base.Engine ({'user_id': 1, 'email_address': 'jack@yahoo.com'}, {'user_id': 1, 'email_address': 'jack@msn.com'}, {'user_id': 2, 'email_address': 'www@www.org'}, {'user_id': 2, 'email_address': 'wendy@aol.com'})
2019-08-18 22:57:29,503 INFO sqlalchemy.engine.base.Engine COMMIT


### selecting

In [26]:
>>> from sqlalchemy.sql import select
>>> s = select([users])
>>> result = conn.execute(s)

2019-08-18 22:57:31,591 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-08-18 22:57:31,592 INFO sqlalchemy.engine.base.Engine {}


In [27]:
>>> for row in result:
...     print(row)

(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


In [28]:
>>> result = conn.execute(s)
>>> row = result.fetchone()
>>> print("name:", row['name'], "; fullname:", row['fullname'])

2019-08-18 22:57:33,435 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-08-18 22:57:33,438 INFO sqlalchemy.engine.base.Engine {}
name: jack ; fullname: Jack Jones


In [29]:
>>> row = result.fetchone()
>>> print("name:", row[1], "; fullname:", row[2])

name: wendy ; fullname: Wendy Williams


In [30]:
>>> for row in conn.execute(s):
...     print("name:", row[users.c.name], "; fullname:", row[users.c.fullname])

2019-08-18 22:57:34,948 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-08-18 22:57:34,950 INFO sqlalchemy.engine.base.Engine {}
name: jack ; fullname: Jack Jones
name: wendy ; fullname: Wendy Williams


In [31]:
result.close()

In [32]:
>>> s = select([users.c.name, users.c.fullname])
>>> result = conn.execute(s)
>>> for row in result:
...     print(row)

2019-08-18 22:57:36,165 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2019-08-18 22:57:36,167 INFO sqlalchemy.engine.base.Engine {}
('jack', 'Jack Jones')
('wendy', 'Wendy Williams')


In [33]:
>>> for row in conn.execute(select([users, addresses])):
...     print(row)

2019-08-18 22:57:37,000 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses
2019-08-18 22:57:37,002 INFO sqlalchemy.engine.base.Engine {}
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(2, 'wendy', 'Wendy Williams', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 2, 1, 'jack@msn.com')
(1, 'jack', 'Jack Jones', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [34]:
>>> for row in conn.execute(select([users])):
...     print(row)

2019-08-18 22:57:38,160 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-08-18 22:57:38,160 INFO sqlalchemy.engine.base.Engine {}
(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


In [35]:
>>> for row in conn.execute(select([addresses])):
...     print(row)

2019-08-18 22:57:38,898 INFO sqlalchemy.engine.base.Engine SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses
2019-08-18 22:57:38,900 INFO sqlalchemy.engine.base.Engine {}
(1, 1, 'jack@yahoo.com')
(2, 1, 'jack@msn.com')
(3, 2, 'www@www.org')
(4, 2, 'wendy@aol.com')


In [36]:
>>> s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
>>> for row in conn.execute(s):
...     print(row)

2019-08-18 22:57:39,544 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-08-18 22:57:39,547 INFO sqlalchemy.engine.base.Engine {}
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [37]:
>>> users.c.id == addresses.c.user_id

In [38]:
str(users.c.id == addresses.c.user_id)

'users.id = addresses.user_id'

In [39]:
print(users.c.id == addresses.c.user_id)

users.id = addresses.user_id


In [40]:
print(users.c.id == 7)

users.id = :id_1


In [41]:
(users.c.id == 7).compile().params

{'id_1': 7}

In [42]:
>>> print(users.c.id != 7)

>>> # None converts to IS NULL
>>> print(users.c.name == None)

>>> # reverse works too
>>> print('fred' > users.c.name)

users.id != :id_1
users.name IS NULL
users.name < :name_1


In [43]:
print(users.c.id + addresses.c.id)

users.id + addresses.id


In [44]:
print(users.c.name + users.c.fullname)

users.name || users.fullname
